In [1]:
import os
from env.audit import Audits

appsecret = os.getenv("admin_appsecret")
appid = os.getenv("admin_appid")
tenantid = os.getenv("admin_tenantid")

audit = Audits()
audit.set_ServicePrincipal(
    tenant_id=tenantid,
    client_id=appid,
    client_secret= appsecret
)


In [2]:
# Welcome to your new notebook
# Type here in the cell editor to add code!

# from notebookutils import mssparkutils
from azure.identity import ClientSecretCredential, DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

LakehouseName = "LochMonitor"
WarehouseName = ""
WorkspaceName = "FabricMonitor"


# must have
audit.set_LakehouseName(LakehouseName)  
audit.set_WorkspaceName(WorkspaceName) 

# optional
# audit.set_ApplicationModules("Activity,Apps,Capacity,Catalog,Domains,FabricItems,Gateway,Graph,Refreshables,RefreshHistory,Tenant,Workspaces")
audit.set_ApplicationModules("Catalog")
audit.set_PathInLakehouse("stage")
audit.set_all_workspaces(True)

audit.exclude_personal_workspaces = True
audit.exclude_inactivate_workspaces = True


# set cron for modules 
# audit.set_Activity_cron("* * * * * 30")
# audit.set_Catalog_cron("* * * * * 30")
# audit.set_Capacity_cron("* * * * * 30")
# audit.set_Apps_cron("* * * * * 30")
# audit.set_Domains_cron("* * * * * 30")
# audit.set_Graph_cron("* * * * * 30")
# audit.set_Tenant_cron("* * * * * 30")
# audit.set_RefreshHistory_cron("* * * * * 30")
# audit.set_Refreshables_cron("* * * * * 30")
# audit.set_Gateway_cron("* * * * * 30")

# Run the application
await audit.run()



what is the current state {"activity": {"lastRun": "2024-10-22T15:04:02.092292Z"}, "apps": {"lastRun": "2024-10-22T15:04:02.092318Z"}, "catalog": {"lastRun": "2024-10-22T12:37:01.659785Z"}, "gateway": {"lastRun": "2024-10-22T15:04:02.092337Z"}, "graph": {"lastRun": "2024-10-22T15:04:02.092340Z"}, "refreshables": {"lastRun": "2024-10-22T15:04:02.092344Z"}, "refreshhistory": {"lastRun": "2024-10-22T15:04:02.092348Z"}, "tenant": {"lastRun": "2024-10-22T15:04:02.092351Z"}, "capacity": {"lastRun": "2024-10-22T15:04:02.092322Z"}, "roles": {"lastRun": "2024-09-23T14:28:41.625610Z"}, "domains": {"lastRun": "2024-10-22T11:18:56.147146Z"}, "fabricitems": {"lastRun": "2024-10-22T11:18:56.147193Z"}, "workspaces": {"lastRun": "2024-10-22T11:18:56.147202Z"}}
Task Catalog Scans (Catalog meta data) is now running
using the following url: admin/workspaces/modified?excludePersonalWorkspaces=True&excludeInActiveWorkspaces=True
Task Catalog Scans (Catalog meta data) is now running
using the following url:

## UPSERT Example

In [ ]:
from delta.tables import DeltaTable

# Assuming you have a Delta table named "my_delta_table"
delta_table_path = "path/to/delta-table"  # Replace with your actual path

# Create or load your DataFrame (myDataFrame) with the data to be upserted
# For demonstration purposes, let's assume you have columns: "Id", "Type", "Value"

# Initialize the DeltaTable
# delta_table = DeltaTable.forPath(spark, delta_table_path)
delta_table = spark.read.format("delta").load(delta_table_path)
myDataFrame = spark.read.format("delta").load(myDataFrame_path)

# Merge the updates into the Delta table
delta_table.alias("events") \
    .merge(myDataFrame.alias("updates"), "events.Id = updates.Id and events.Type = updates.Type") \
    .whenMatchedUpdateAll() \
    .whenNotMatchedInsertAll() \
    .execute()

# Commit the changes
delta_table.toDF().write.format("delta").mode("overwrite").save(delta_table_path)


In [ ]:
from delta.tables import DeltaTable

# Assuming delta_table and myDataFrame are already defined
# delta_table_path = "path/to/delta-table"  # Replace with your actual path

# Initialize the DeltaTable
delta_table = DeltaTable.forPath(spark, delta_table_path)

# Merge the updates into the Delta table using datasetId for comparison
delta_table.alias("existing") \
    .merge(
        myDataFrame.alias("updates"),
        "existing.datasetId = updates.datasetId"
    ) \
    .whenMatchedUpdateAll() \
    .whenNotMatchedInsertAll() \
    .execute()

# Commit the changes
delta_table.toDF().write.format("delta").mode("overwrite").save(delta_table_path)

In [ ]:
from pyspark.sql import SparkSession 
from pyspark.sql.types import StructType, StructField, StringType, TimestampType
from pyspark.sql.functions import lit
from pyspark import SparkConf


In [ ]:
from dataclasses import dataclass, fields, InitVar

@dataclass
class Rectangle:
    width: int
    height: int

    def area(self):
        return self.width * self.height

    def perimeter(self):
        return 2 * (self.width + self.height)
    
@dataclass
class ColoredRectangle(Rectangle):
    color: str


@dataclass
class C:
    i: int
    j: int | None = None
    database: InitVar[str|None] = None

    def __post_init__(self, database):
        if self.j is None and database is not None:
            self.j = database.lookup("j")


c = C(10, database={"j": "value"})
print(c.j)
rect = ColoredRectangle(3, 4, 'blue')

print(rect.area())
print(rect.color)

In [ ]:
def is_key_nested(d, key):
    if key in d:
        return True
    for k, v in d.items():
        if isinstance(v, dict):
            if is_key_nested(v, key):
                return True
        elif isinstance(v, list):
            for item in v:
                if isinstance(item, dict) and is_key_nested(item, key):
                    return True
    return False

is_datasets_nested = is_key_nested(myDict, 'datasets')
print(is_datasets_nested)


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchIndex, SimpleField, SearchFieldDataType
from azure.core.credentials import AzureKeyCredential
from langchain.embeddings import OpenAIEmbeddings

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# Read the binary file
file_path = "path/to/your/binary/file"
with open(file_path, "rb") as file:
    binary_content = file.read()

# Convert binary content to text (assuming it's text-based binary like PDF)
text_content = binary_content.decode('utf-8')

# Split the text content into chunks
chunks = text_splitter.split_text(text_content)

# Azure Search service details
search_service_name = "your-search-service-name"
index_name = "your-index-name"
admin_key = "your-admin-key"

# Create a SearchClient
search_client = SearchClient(
    endpoint=f"https://{search_service_name}.search.windows.net",
    index_name=index_name,
    credential=AzureKeyCredential(admin_key)
)

# Insert chunks into Azure AI Search Index
for i, chunk in enumerate(chunks):
    document = {
        "id": str(i),
        "content": chunk
    }
    search_client.upload_documents(documents=[document])

print("Chunks inserted into Azure AI Search Index successfully.")
# Initialize the OpenAI embeddings
embeddings = OpenAIEmbeddings()

# Generate embeddings for each chunk
embedded_chunks = [embeddings.embed(chunk) for chunk in chunks]

# Insert embedded chunks into Azure AI Search Index
for i, (chunk, embedding) in enumerate(zip(chunks, embedded_chunks)):
    document = {
        "id": str(i),
        "content": chunk,
        "embedding": embedding
    }
    search_client.upload_documents(documents=[document])

print("Chunks with embeddings inserted into Azure AI Search Index successfully.")

In [ ]:
for key in myDict:
    print(myDict[key])
    if isinstance(myDict[key], dict):
        for k in myDict[key]:
            print(k)
    if isinstance(myDict[key], list):
        for i in range(0, len(myDict[key])):
            for j in myDict[key][0]:
                print(myDict[key][0][j])


In [ ]:
sparkConf = SparkConf()
sparkConf.set("fs.azure.account.auth.type.onelake.dfs.core.windows.net", "OAuth")
sparkConf.set("fs.azure.account.oauth.provider.type.onelake.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
sparkConf.set("fs.azure.account.oauth2.client.id.onelake.dfs.core.windows.net", appid)
sparkConf.set("fs.azure.account.oauth2.client.secret.onelake.dfs.core.windows.net", appsecret)
sparkConf.set("fs.azure.account.oauth2.client.endpoint.onelake.dfs.core.windows.net", "https://login.microsoftonline.com/" + tenantid)
sparkConf.set("spark.jars.packages", "org.apache.hadoop:hadoop-azure:3.3.1")
spark = SparkSession.builder.config(conf=sparkConf).appName("SparkTest").getOrCreate()


schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", StringType(), True),
    StructField("timestamp", TimestampType(), True)
])

import datetime

now = datetime.datetime.now()

data = [("John", "30", now), ("Jane", "25", now)]
df = spark.createDataFrame(data, schema)
df = df.withColumn("source", lit("test"))

df.write.format("parquet").mode("append").save("abfss://onelake.dfs.core.windows.net/FabricLake.Lakehouse/Files/data.parquet")
# df.write.format("parquet").mode("append").save("FabricLake.Lakehouse/Files/data.parquet")
df.show()



In [ ]:
# df.write.format("delta").mode("append").saveAsTable("data")
df.write.format("parquet").save("data.parquet")

In [ ]:
# Instantiate a DataLakeServiceClient using a connection string
   from azure.storage.filedatalake import DataLakeServiceClient
   datalake_service_client = DataLakeServiceClient.from_connection_string(self.connection_string)

   # Instantiate a FileSystemClient
   file_system_client = datalake_service_client.get_file_system_client("mynewfilesystem")

In [ ]:
from time import sleep
import datetime


counter = int()

for i in range(1,84):
    counter+=1

    if i % 17 == 0:
        print(datetime.datetime.now())
        sleep(5)
        print(f"FizzBuzz and {counter}")
        # print(datetime.datetime.now())


In [ ]:
# t = range(1,15000)

def distribute_items(items, num_groups):
    # Calculate the size of each group
    group_size = len(items) // num_groups
    remainder = len(items) % num_groups

    # Create the groups
    groups = []
    start = 0
    for i in range(num_groups):
        end = start + group_size + (1 if i < remainder else 0)
        groups.append(items[start:end])
        start = end

    return groups

# Example usage
items = list(range(1, 15001))  # List of 15,000 items
num_groups = 16
groups = distribute_items(items, num_groups)

# Print the size of each group
for i, group in enumerate(groups):
    print(f"Group {i+1}: {len(group)} items")


In [ ]:
from azure.identity import DefaultAzureCredential
from azure.storage.filedatalake import DataLakeServiceClient, FileSystemClient
from azure.identity import ClientSecretCredential
import datetime
import json

storage_account_name = "onelake"
workspace_name = "FabricMonitor"

credential = ClientSecretCredential(tenant_id=tenantid,
                            client_id=appid,
                            client_secret=appsecret)

file_system_client = FileSystemClient(
    account_url="https://onelake.dfs.fabric.microsoft.com",
    file_system_name=workspace_name,
    credential=credential)

dc = file_system_client.get_directory_client("FabricLake.Lakehouse/Files")
file_client = dc.get_file_client("data.parquet")
with open("./data.parquet", "rb") as data:
    file_client.upload_data(data, overwrite=True)

    


df.write.format("delta").mode("append").save("abfss://onelake.dfs.core.windows.net/FabricLake.Lakehouse/Files/data.parquet")


In [ ]:
import pandas as pd
# Load data into pandas DataFrame from "/lakehouse/default/" + "Files/stage/state.json"
state = pd.read_json("/lakehouse/default/" + "Files/stage/state.yaml",typ="series")

lastRun = state['activity']['lastRun']
lastRun


In [ ]:
with open("/lakehouse/default/" + "Files/stage/state.yaml", "r") as file:
    for line in file:
        print(line)

In [ ]:
s = spark.read.option("multiline", "true").json("Files/stage/state.json")
df = spark.read.option("multiLines","json").json("Files/stage/catalog/scans/2024/07/22/*.json")

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from functools import reduce

data = df.withColumn("datasets", explode(df["workspaces"]["datasets"]))

df = df.withColumn("start_Time", to_utc_timestamp(df['startTime'], "UTC"))
df = df.withColumn("cst_timestamp", from_utc_timestamp(df.start_Time, "America/Chicago"))
df = df.withColumn("eu_timestamp", from_utc_timestamp(df.start_Time, "Europe/Berlin"))
df = df.withColumn("ist_timestamp", from_utc_timestamp(df.start_Time, "Asia/Kolkata"))


In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from functools import reduce
from  pyspark.sql.functions import input_file_name

dc = df.withColumn("requestId", df.requestId)\
    .withColumn("refreshAttempts", explode(df.refreshAttempts))\
    .select("requestId","refreshAttempts")

display(dc)

In [ ]:
de = dc.withColumn("attemptId", dc.refreshAttempts.attemptId)\
    .withColumn("startTime", dc.refreshAttempts.startTime)\
    .withColumn("endTime", dc.refreshAttempts.endTime)\
    .withColumn("type", dc.refreshAttempts.type)\
    .select("requestId","attemptId","startTime","endTime","type")

display(de)

In [ ]:
df.select('dataset_id', 'dataset_name', 'endTime', 'id', 'refreshType', 'requestId', 'serviceExceptionJson', 'startTime', 'status').dropDuplicates(["dataset_id","requestId"]).write.format("delta").mode("overwrite").option("overwriteSchema","true").save("Tables/refreshhistory")
de.write.format("delta").mode("overwrite").option("overwriteSchema","true").save("Tables/refreshhistory_attempts")

In [ ]:
%%sql

SELECT dataset_id, COUNT(DISTINCT requestId) as requestId_count
FROM refreshhistory
GROUP BY dataset_id
HAVING COUNT(DISTINCT requestId) > 1


### Get the latest record by date

In [ ]:
ds = "Tables/workspaces_workspaces"
df = spark.read.format("delta").load(ds)
dd = df.withColumn("scandate", concat_ws("-",df.ts_year, df.ts_month, df.ts_day).cast("date"))

# Define window specification
window_spec = Window.partitionBy("id").orderBy(col("scandate").desc())

# Add row number to each partition
df_with_row_num = dd.withColumn("row_number", row_number().over(window_spec))

# Filter to get the latest records for each id
latest_records_df = df_with_row_num.filter(col("row_number") == 1).drop("row_number")

# Show the latest records DataFrame
latest_records_df.write.format("delta").mode("overwrite").option("overwriteSchema","true").save(ds)

In [ ]:
df = spark.read.option("multiline","true").json("Files/stage/catalog/scans/2024/*/*/*.json")
dd = df.withColumn("workspaces", explode(df.workspaces)).drop("datasourceInstances")

workspaces = dd.withColumn("capacityId", dd["workspaces"]["capacityId"]) \
    .withColumn("dataRetrievalState", dd["workspaces"]["dataRetrievalState"]) \
    .withColumn("defaultDatasetStorageFormat", dd["workspaces"]["defaultDatasetStorageFormat"]) \
    .withColumn("description", dd["workspaces"]["description"]) \
    .withColumn("domainId", dd["workspaces"]["domainId"]) \
    .withColumn("id", dd["workspaces"]["id"]) \
    .withColumn("isOnDedicatedCapacity", dd["workspaces"]["isOnDedicatedCapacity"]) \
    .withColumn("name", dd["workspaces"]["name"]) \
    .withColumn("state", dd["workspaces"]["state"]) \
    .withColumn("type", dd["workspaces"]["type"]) 


ds = dd.withColumn("id", dd.workspaces.id)\
    .withColumn("datasets", explode(dd.workspaces.datasets)).drop("workspaces")

ds = ds.withColumn("dataset_id", ds["datasets"]["id"])

ds.select("id","dataset_id").dropDuplicates().write.format("delta").mode("overwrite").save("Tables/catalog_workspace_datasets")

ee = spark.read.format("delta").load("Tables/datasetrefreshable_datasetrefreshable")
display(ee)

workspaces = dd.withColumn("id", dd["workspaces"]["id"]) \
            .withColumn("defaultDatasetStorageFormat",dd["workspaces"]["defaultDatasetStorageFormat"])\
            .withColumn("description",dd["workspaces"]["description"])\
            .withColumn("domainId",dd["workspaces"]["domainId"])\
            .withColumn("isOnDedicatedCapacity",df["workspaces"]["isOnDedicatedCapacity"])\
            .withColumn("datasets",explode(df["workspaces"]["datasets"]))

dd = df.withColumn("workspaces", explode(df.workspaces)).drop("datasourceInstances")
display(dd)

display(dd.select(dd["workspaces"]["id"]))

df = spark.read.format("delta").load("Tables/catalog_datasets")
display(df.limit(10))

datasets = df.withColumn("scandate", concat_ws("-",df.ts_year, df.ts_month, df.ts_day).cast("date"))
display(datasets.limit(10))

In [ ]:
ds = "Tables/workspaces_workspaces"
df = spark.read.format("delta").load(ds)
dd = df.withColumn("scandate", concat_ws("-",df.ts_year, df.ts_month, df.ts_day).cast("date"))

# Define window specification
window_spec = Window.partitionBy("id").orderBy(col("scandate").desc())

# Add row number to each partition
df_with_row_num = dd.withColumn("row_number", row_number().over(window_spec))

# Filter to get the latest records for each id
latest_records_df = df_with_row_num.filter(col("row_number") == 1).drop("row_number")

# Show the latest records DataFrame
latest_records_df.write.format("delta").mode("overwrite").option("overwriteSchema","true").save(ds)

latest_records_df.write.format("delta").save("Tables/catalog_datasets_unique")

df = spark.read.format("delta").load("Tables/catalog_datasets_tables")
df_tables = df.withColumn("scandate", concat_ws("-",df.ts_year, df.ts_month, df.ts_day).cast("date"))

df = spark.read.format("delta").load("Tables/catalog_datasets_columns")
df_columns = df.withColumn("scandate", concat_ws("-",df.ts_year, df.ts_month, df.ts_day).cast("date"))

df = spark.read.format("delta").load("Tables/catalog_datasets_measures")
df_measures = df.withColumn("scandate", concat_ws("-",df.ts_year, df.ts_month, df.ts_day).cast("date"))



In [ ]:
measures = df_measures.withColumn("UID", concat_ws("-", col("datasets_id"), col("datasets_tables_name"), col("scandate")))
tables = df_tables.withColumn("UID", concat_ws("-", col("datasets_id"), col("datasets_tables_name"), col("scandate")))
columns = df_columns.withColumn("UID", concat_ws("-", col("datasets_id"), col("datasets_tables_name"), col("scandate")))

measures.write.format("delta").save("Tables/catalog_datasets_measures_history")
tables.dropDuplicates("UID").write.format("delta").save("Tables/catalog_datasets_tables_history")
columns.write.format("delta").save("Tables/catalog_datasets_columns_history")

In [ ]:
dd = spark.read.format("delta").load("Tables/catalog_datasets_columns_history")
display(dd.limit(10))

In [ ]:
# tables = spark.read.format("delta").load("Tables/catalog_datasets_tables_history")
cols = ["datasets_id","datasets_tables_name","scandate"]
display(tables.orderBy(*cols).select(*cols))

In [ ]:
dd.dropDuplicates(["UID"]).write.format("delta").mode("overwrite").option("overwriteSchema", "True").save("Tables/catalog_datasets_tables_history")

In [ ]:
%%sql

select T.UID, datasets_tables_name, datasets_id, scandate, description, name FROM catalog_tables_history as T
INNER JOIN (select UID, COUNT(UID) FROM catalog_tables_history group by UID having count(UID)>1) as G
ON T.UID=G.UID

In [ ]:
cols = ["datasets_id", "datasets_tables_name"]

window_spec = Window.partitionBy(*cols).orderBy(col("scandate").desc())

# Add row number to each partition
df_with_row_num = df_tables.withColumn("row_number", row_number().over(window_spec))

# Filter to get the latest records for each id
latest_records_df = df_with_row_num.filter(col("row_number") == 1).drop("row_number")

# Show the latest records DataFrame
latest_records_df.select(col("datasets_id"), col("datasets_tables_name"), col("scandate")).show()